**Hidden Markov models for cracking codes**

In this exercise you have to make a partially built HMM work and use it to solve some simple substitution ciphers. Plaintext data is provided in 'plaintext' directory. Encrypted data is in 'encrypted'. Some of the texts were originally English some of them were Russian; the sequences are also of different lengths. 

This homework is worth **15 points** and is due by the next class (**24th Oct.**), please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 'filename author' where 'filename' is a file from "encrypted/\*_encrypted.txt" and 'author' is a file from "plaintext/\*.txt" (not including 'english.txt', 'russian.txt' or 'all.txt') which best matches the decrypted text.




In [1]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in char_to_int_mapping.items()]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))
    move_out_count = np.zeros(num_states)

    for n, sequence in enumerate(sequences):
        for prev, nxt in zip(sequence[:-1], sequence[1:]):
            A_counts[prev, nxt] += 1
            pi_counts[prev] += 1
        if len(sequence) > 0:
            pi_counts[sequence[-1]] += 1
            
    pi = pi_counts / np.sum(pi_counts)
    A = A_counts / np.sum(A_counts, axis=1)[:, None]
    
    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [2]:
%%time
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/1_encrypted.txt' # short sequences in english
# ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))


CPU times: user 5.4 s, sys: 4.25 ms, total: 5.4 s
Wall time: 5.4 s


Below is a mostly implemented HMM.

In [3]:
from IPython.display import clear_output

In [4]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}, max_len=15):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping) # number of states = number of chars 
        self.num_outputs = len(observations_to_char_mapping) # number of outputs ?
        self.states_to_char_mapping = states_to_char_mapping # get char from state
        self.observations_to_char_mapping = observations_to_char_mapping # what is observations ??? 
        
        self.max_len = max_len ## MAX LEN OF ENCRIPTED TEXT
        # Random initialization
        self.pi = np.random.rand(self.num_states)# init pi with random 
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states) # init transition matrix with random 
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs) # init emission matrix 
        self.B /= np.sum(self.B, 1, keepdims=True)
        
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        self.pi = parameters['pi'] if self.fixed_pi else self.pi 
        
        self.fixed_A = 'A' in parameters
        self.A = parameters['A'] if self.fixed_A else self.A
        
        self.fixed_B = 'B' in parameters
        self.B = parameters['B'] if self.fixed_B else self.B
           
        previous_llh = None # prev log-likelihood
        
        iteration = 0
        while True and iteration < max_iters: # why we dont use for-loop here? 
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences) # E-step from EM-algo

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts) # M-step from EM-algo

            # Output some sequences for debugging.
            if iteration % 10 == 0:
                # clear_output()
                print('iteration %d; log likelihood %.4f' % (iteration, log_likelihood))
                # self.output(sequences[:10])

            # Log likelihood should be increasing
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iteration += 1
        return previous_llh, self.output(sequences[:10])

    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            for i in range(0, len(sequence), self.max_len):
                sub_seq = sequence[i:i+self.max_len] 

                # Run Forward-Backward dynamic program
                alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sub_seq)

                # Accumulate statistics.
                pi_counts += gamma[:, 0]
                A_counts += xi
                for t, x in enumerate(sub_seq):
                    B_counts[:, x] += gamma[:, t]

                total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = self.forward(sequence) 
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        ss = np.sum(alpha * beta, 0)
        if (ss == 0).any():
            print('err')
        assert np.isfinite(ss).all()
        gamma = (alpha * beta) / ss

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        if not self.fixed_A:
            for t in range(1, len(sequence)-1):
                this_xi = np.zeros_like(self.A) ## Changed for better performance 
                this_xi += (self.A * alpha[:, t][:, None]) * (beta[:, t+1] * self.B[:, sequence[t+1]])      
                xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1]))

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        alpha[:, 0] = self.pi * self.B[:, sequence[0]]
        for t in range(len(sequence) - 1):
            x = sequence[t+1]
            alpha[:, t+1] = self.B[:, x] * np.sum(self.A * alpha[:, t][:, None], axis=0)
        return alpha 
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        beta[:, -1] = 1
        
        for i, x in enumerate(sequence[1:][::-1]):
            t = len(sequence) - 1 - i
            beta[:, t-1] = np.sum(self.A * (beta[:, t] * self.B[:, x]), axis=1)
        
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        res = []
        for i, sequence in enumerate(sequences):
            observations = []
            map_states = []
            for i in range(0, len(sequence), self.max_len):
                sub_seq = sequence[i:i+self.max_len] 
                sub_observations = [self.observations_to_char_mapping[x] for x in sub_seq]                
                sub_map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sub_seq)]
                observations.append(''.join(sub_observations))
                map_states.append(''.join(sub_map_states))
                
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))
            res.append('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))
        return res
    
    
    def output_decripted(self, sequences):
        # Output some decoded states. 
        res = []
        for i, sequence in enumerate(sequences):
            observations = []
            map_states = []
            for i in range(0, len(sequence), self.max_len):
                sub_seq = sequence[i:i+self.max_len] 
                sub_map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sub_seq)]
                map_states.append(''.join(sub_map_states))
            res.append(''.join(map_states) + '\n')
        return res


**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [5]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping, max_len=15)

# Estimate the parameters and decode the encrypted sequences.
llh, output = hmm.estimate_with_em(encrypted_sequences[:100], parameters={'pi': pi, 'A': A})

iteration 0; log likelihood -12081.8386
iteration 10; log likelihood -9925.3214
iteration 20; log likelihood -9558.5769
iteration 30; log likelihood -9474.5125
iteration 40; log likelihood -9427.6650
iteration 50; log likelihood -9381.4015
iteration 60; log likelihood -9340.9561
iteration 70; log likelihood -9276.4958
iteration 80; log likelihood -9113.3362
iteration 90; log likelihood -8902.4124
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cancous britis bait fot me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill fot endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prantice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       britis go to you are fot cassius
(observations):

### Task2 results 
We can see that hmm works good enough with small english texts

p.s. this is work of algorithm with task3-4 improvements

**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/99_encrypted.txt' (note these are in Russian).

In [8]:
plaintext_russian = 'plaintext/russian.txt'
ciphertext_russian = 'encrypted/99_encrypted.txt'

char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext_russian)

plaintext_sequences = load_sequences(plaintext_russian, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext_russian, char_to_int_mapping)
pi_ru, A_ru = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))



state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
hmm = HMM(observation_to_char_mapping, state_to_char_mapping, max_len=30)

llh, output = hmm.estimate_with_em(encrypted_sequences[:30], parameters={'pi': pi_ru, 'A': A_ru}, max_iters=100)
# take first 50 encripted examples to reduce time of processing
# for the same reason change the number of iterations

iteration 0; log likelihood -22976.0006
iteration 10; log likelihood -16703.9203
iteration 20; log likelihood -15969.2359
iteration 30; log likelihood -15550.6683
iteration 40; log likelihood -15500.7274
iteration 50; log likelihood -15490.3822
iteration 60; log likelihood -15487.1750
iteration 70; log likelihood -15486.0711
iteration 80; log likelihood -15485.4633
iteration 90; log likelihood -15485.0901
(states):       князь андрей пожал плечами и поморщился как морщатся любители музыки услышав пальшивую ноту обе женщины отрустили друг друга потом опять как бучто боясь опоздать схватили друг друга за руки стали целовать и отрывать руки и потом опять стали целовать друг друга в лицо и совершенно неожиданно для князя андрея обе 194091907 и опять стали целоваться тоже 1940919ла князю андрей было очевидно неловко но для двух женщий казалось так естественно что они плакали казалось они и не предполагали чтобы могло иначе совершиться это свидание
(observations): йдрьуопдшщмзояю4пфояфм3пч о

### Task3 Results
- Hmm works quite good for that section but we have some artifacts
- And also we can guess from that text, that the book is "War and piece"

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

### Task4 Results

Already did
- Save just outermost for-loop for `forward` and `backward` function. 
- Optimize compute `xi` for-loop at `forward_backward` function. 

## Done 

**TASK 5**: Try to classify the author of each text. 

## Plan
For each in texts russian and texts in english try to decode file. 

For each decoded files find original, that best matches (most lines are matched)


In [15]:
def get_russian():
    russian_list = []
    for i in range(143):
        try:
            ciphertext_find = 'encrypted/{}_encrypted.txt'.format(i)
            encrypted_sequences_tt = load_sequences(ciphertext_find, char_to_int_mapping_tt)
            russian_list.append(i)
        except:
            continue
    return russian_list

In [17]:
rl = get_russian()


In [82]:
import pickle

with open('res_rus.pkl', 'wb') as f:
    pickle.dump(rr, f)

In [104]:
import numpy as np

In [9]:
def decode_text(plain_file, cipher_files, out_files, iters=41, max_len=15):
    scores = []
    
    char_to_int_mapping_tt, int_to_char_mapping_tt = get_char_to_int_mapping(plain_file)
    plaintext_sequences_tt = load_sequences(plain_file, char_to_int_mapping_tt)
    pi_tt, A_tt = estimate_markov_model_from_sequences(plaintext_sequences_tt, len(char_to_int_mapping_tt))
        
    for i, cipher_file in enumerate(cipher_files):
        print(cipher_file)
        ciphertext_find = cipher_file # longer sequences in russian
        encrypted_sequences_tt = load_sequences(ciphertext_find, char_to_int_mapping_tt)
        
        state_to_char = int_to_char_mapping_tt
        observation_to_char = int_to_char_mapping_tt

        hmm_tt = HMM(observation_to_char, state_to_char, max_len=max_len)

        llh, res = hmm_tt.estimate_with_em(encrypted_sequences_tt[:200],
                                           parameters={'pi': pi_tt, 'A': A_tt}, max_iters=iters)
        with open(out_files[i], 'w', encoding='utf8') as f:
            f.writelines(hmm_tt.output_decripted(encrypted_sequences_tt))
        clear_output()
    return scores

In [18]:
el = [i for i in range(143) if i not in rl]

In [20]:
rr = []
enc_files = []
dec_files = []
for num in rl:
    enc_files.append('encrypted/{}_encrypted.txt'.format(num))
    dec_files.append('decripted2/{}_decrypted.txt'.format(num))
    
decode_text('plaintext/russian.txt', enc_files, dec_files, iters=100)
print('done')

done


In [205]:
res = {
    'plaintext/evgeniy-onegin.txt': {},
    'plaintext/khlebnikov.txt': {},
    'plaintext/mayakovsky.txt': {},
    'plaintext/voina-i-mir-tom-1.txt': {},
    'plaintext/voina-i-mir-tom-2.txt': {}
}

for author in res.keys():
    with open(author, 'r') as f:
        auth_file = f.read()
        
    for dec_file in dec_files:
        print(dec_file)
        with open(dec_file, 'r') as f1:
            dec_lines = f1.readlines()
        counts = 0 
        for line in dec_lines:
            if line in auth_file:
                counts += 1
        if counts > 0:
            res[author][dec_file] = counts
        
    clear_output()

In [210]:
res_ru = {}
i = 0
for auth in res.keys():
    for dec_file in res[auth].keys():
        if dec_file not in res_ru:
            res_ru[dec_file] = [0,0,0,0,0,0,0]
        res_ru[dec_file][i] = res[auth][dec_file]
    i += 1

In [211]:
res_ru

{'decripted2/103_decrypted.txt': [720, 0, 0, 0, 1, 0, 0],
 'decripted2/105_decrypted.txt': [33, 505, 51, 25, 30, 0, 0],
 'decripted2/107_decrypted.txt': [14, 17, 17, 12, 11, 0, 0],
 'decripted2/108_decrypted.txt': [11, 14, 20, 10, 9, 0, 0],
 'decripted2/110_decrypted.txt': [5, 5, 7, 9, 432, 0, 0],
 'decripted2/11_decrypted.txt': [59, 64, 566, 52, 46, 0, 0],
 'decripted2/123_decrypted.txt': [33, 36, 34, 33, 33, 0, 0],
 'decripted2/128_decrypted.txt': [14, 15, 30, 12, 11, 0, 0],
 'decripted2/129_decrypted.txt': [56, 65, 594, 54, 53, 0, 0],
 'decripted2/135_decrypted.txt': [3, 4, 4, 396, 6, 0, 0],
 'decripted2/137_decrypted.txt': [4, 4, 4, 4, 4, 0, 0],
 'decripted2/139_decrypted.txt': [0, 1, 1, 0, 0, 0, 0],
 'decripted2/140_decrypted.txt': [62, 56, 643, 45, 54, 0, 0],
 'decripted2/14_decrypted.txt': [38, 699, 32, 27, 26, 0, 0],
 'decripted2/17_decrypted.txt': [737, 1, 1, 1, 1, 0, 0],
 'decripted2/19_decrypted.txt': [2, 2, 3, 6, 533, 0, 0],
 'decripted2/26_decrypted.txt': [12, 19, 15, 10, 

In [212]:
with open('ru_res', 'wb') as f:
    pickle.dump(res, f)

In [213]:
dec_enc_map = {dec:enc for dec, enc in  zip(dec_files, enc_files)} 

In [214]:
files = [
    'plaintext/evgeniy-onegin.txt',
    'plaintext/khlebnikov.txt',
    'plaintext/mayakovsky.txt',
    'plaintext/voina-i-mir-tom-1.txt',
    'plaintext/voina-i-mir-tom-2.txt'
]

In [220]:
out = []
for dec in dec_files:
    if dec not in res_ru:
        out.append('{} -------\n'.format(dec_enc_map[dec]))
        continue
    
    i = np.argmax(res_ru[dec])
    
    if res_ru[dec][i] > 50:
        out.append('{} {}\n'.format(dec_enc_map[dec], files[i]))
    else:
        out.append('{} -------\n'.format(dec_enc_map[dec]))



In [221]:
out

['encrypted/5_encrypted.txt plaintext/mayakovsky.txt\n',
 'encrypted/11_encrypted.txt plaintext/mayakovsky.txt\n',
 'encrypted/14_encrypted.txt plaintext/khlebnikov.txt\n',
 'encrypted/17_encrypted.txt plaintext/evgeniy-onegin.txt\n',
 'encrypted/19_encrypted.txt plaintext/voina-i-mir-tom-2.txt\n',
 'encrypted/26_encrypted.txt -------\n',
 'encrypted/27_encrypted.txt -------\n',
 'encrypted/32_encrypted.txt plaintext/khlebnikov.txt\n',
 'encrypted/62_encrypted.txt -------\n',
 'encrypted/63_encrypted.txt -------\n',
 'encrypted/64_encrypted.txt plaintext/khlebnikov.txt\n',
 'encrypted/80_encrypted.txt -------\n',
 'encrypted/86_encrypted.txt -------\n',
 'encrypted/97_encrypted.txt -------\n',
 'encrypted/99_encrypted.txt -------\n',
 'encrypted/103_encrypted.txt plaintext/evgeniy-onegin.txt\n',
 'encrypted/105_encrypted.txt plaintext/khlebnikov.txt\n',
 'encrypted/107_encrypted.txt -------\n',
 'encrypted/108_encrypted.txt -------\n',
 'encrypted/110_encrypted.txt plaintext/voina-i-mi

In [222]:
with open('result_ru.txt', 'w') as f:
    f.writelines(out)